In [ ]:
### 🛒 Desafio Extra — Projeto de Vendas com PostgreSQL usando psycopg2 e ipython-sql.

Agora que você finalizou a análise dos dados de vendas, chegou o momento de estruturar tudo como um banco de dados relacional real!

Neste desafio, você irá:

✅ Conectar ao PostgreSQL  
✅ Criar as tabelas via SQL  
✅ Inserir os dados linha a linha a partir do DataFrame (o dataframe gerado da Venda o qual calculamos a venda com aumento) .
✅ Realizar algumas consultas de negócio com SQL puro  

---

#### 🎯 Objetivo

Transformar os dados do seu projeto de vendas em um banco PostgreSQL, criando uma tabela chamada `vendas_final`, contendo as informações abaixo:

##### 🧱 Estrutura esperada da tabela `vendas_final`

| Campo           | Tipo     | Descrição                            |
|-----------------|----------|--------------------------------------|
| cod_produto     | texto    | Código identificador do produto      |
| nome_produto    | texto    | Nome do produto                      |
| categoria       | texto    | Categoria do produto                 |
| valor_venda     | numérico | Valor original da venda              |
| venda_final     | numérico | Valor com aumento aplicado           |
| nome_vendedor   | texto    | Nome do vendedor responsável         |
| data_venda      | data     | Data em que ocorreu a venda          |


# Tentativa de conexão ao banco PostgreSQL
try:
    conn = psycopg2.connect(
        host="localhost",
        database="ProjetoVendas",  # Nome atualizado do banco
        user="postgres",              # Altere se necessário
        password="1234"               # Altere se necessário
    )
    print("Conexão bem sucedida!")
except Exception as e:
    print("Erro ao conectar ao banco de dados:", e)

# Criando o cursor
crsr = conn.cursor()

```
#### 🔑 Etapa 2 — Crie as tabelas com SQL com Python;
#### 🔎 Etapa 3 - Inserir os dados na tabela apartir do dataframe.
#### 📥 Etapa 4 - Realizar algumas consultas de negócio com SQL puro.

- Qual foi o total de vendas por categoria?
- Quem foram os 3 vendedores que mais venderam?
- Qual a média de valor final de venda?


#### 📦 Entregáveis do Desafio:

✅ Notebook .ipynb com:

- Conexão via psycopg2;
- Criação da tabela vendas_final;
- Inserção linha a linha dos dados;
- Banco PostgreSQL populado com os dados corretos;
- Execução de 3 consultas SQL obrigatórias;

✅ Entrega do Projeto:

- Subir o projeto no GITHUB, documentado e me enviar o link do repositório por email.
- Ou enviar o projeto por e-mail: nayara.valevskii@gmail.com

#### DICA:

Aproveite o projeto para o portfólio, documente e post no Linkedin.

Boa sorte, galerinhaaa! 🔥

In [42]:
# Importação de bibliotecas necessárias
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_batch
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:

#### 🔌 Etapa 1 — Conectando ao banco PostgreSQL

In [43]:
# ## Etapa 1: Conexão com o Banco de Dados

# %%
def conectar_banco():
    """Estabelece conexão com o banco PostgreSQL"""
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="ProjetoVendas",
            user="postgres",
            password="1234"
        )
        print("Conexão estabelecida com sucesso!")
        return conn
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

# Testando a conexão
conn = conectar_banco()

Conexão estabelecida com sucesso!


In [ ]:
## Etapa 2: Processamento dos Dados

In [51]:

# Carregando os dados da venda com aumento
df = pd.read_csv('C:\\Users\\soray\\Downloads\\Aula 4\\base_vendas.csv', encoding='ISO-8859-1')  # Codificação Latin-1


In [53]:
try:
    # Carrega os dados com codificação alternativa
    df_vendas = pd.read_csv('base_vendas.csv', sep=';', encoding='ISO-8859-1')
    df_categorias = pd.read_csv('categorias_valores.csv', encoding='ISO-8859-1')
    
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar os dados: {e}")

Dados carregados com sucesso!


In [55]:
try:
    # Carrega os dados de vendas
    df_vendas = pd.read_csv('base_vendas.csv', sep=';', encoding='ISO-8859-1')

    # Carrega os multiplicadores de categoria
    df_categorias = pd.read_csv('categorias_valores.csv', encoding='ISO-8859-1')

    # Visualização dos dados
    print("\nPrimeiras linhas dos dados de vendas:")
    print(df_vendas.head())

    print("\nMultiplicadores por categoria:")
    print(df_categorias)

except Exception as e:
    print(f"Erro ao carregar os arquivos: {e}")



Primeiras linhas dos dados de vendas:
   cod_produto      nome_produto categoria_produto segmento_produto  \
0  SKU-0000001   LG K10 TV Power         Celulares      Corporativo   
1  SKU-0000002  Geladeira Duplex  Eletrodomésticos        Doméstico   
2  SKU-0000003    Lavadora 11 Kg  Eletrodomésticos        Doméstico   
3  SKU-0000004    Lavadora 11 Kg  Eletrodomésticos        Doméstico   
4  SKU-0000005    Lavadora 11 Kg  Eletrodomésticos        Doméstico   

  marca_produto  cod_vendedor     nome_vendedor cod_loja cidade_loja  \
0            LG          1009      Ana Teixeira   SP8821   São Paulo   
1      Brastemp          1006      Josias Silva   SP8821   São Paulo   
2      Brastemp          1006      Josias Silva   SP8821   São Paulo   
3      Brastemp          1003  Mateus Gonçalves   SP8821   São Paulo   
4    Electrolux          1004     Artur Moreira   SP8821   São Paulo   

  estado_loja  data_venda  valor_venda  
0   São Paulo  04/10/2012       679.00  
1   São Paulo  01/0

In [56]:
# Converter a coluna de data para o formato datetime
df_vendas['data_venda'] = pd.to_datetime(df_vendas['data_venda'], dayfirst=True)

In [57]:
# Criar um dicionário com os multiplicadores de categoria
multiplicadores = dict(zip(df_categorias['Categoria'], df_categorias['Valor']))


In [58]:
# Aplicar os multiplicadores para calcular o valor final
df_vendas['venda_final'] = df_vendas.apply(
    lambda row: row['valor_venda'] * multiplicadores.get(row['categoria_produto'], 1),
    axis=1
)

In [59]:
# Selecionar apenas as colunas necessárias para o banco de dados
df_final = df_vendas[[
    'cod_produto', 'nome_produto', 'categoria_produto', 
    'valor_venda', 'venda_final', 'nome_vendedor', 'data_venda'
]]

In [60]:
# Renomear colunas para padronização
df_final.columns = [
    'cod_produto', 'nome_produto', 'categoria', 
    'valor_venda', 'venda_final', 'nome_vendedor', 'data_venda'
]

In [61]:
# Verificar o resultado
print("\nDataFrame final para inserção no banco:")
print(df_final.head())


DataFrame final para inserção no banco:
   cod_produto      nome_produto         categoria  valor_venda  venda_final  \
0  SKU-0000001   LG K10 TV Power         Celulares       679.00       814.80   
1  SKU-0000002  Geladeira Duplex  Eletrodomésticos       832.00       832.00   
2  SKU-0000003    Lavadora 11 Kg  Eletrodomésticos       790.00       790.00   
3  SKU-0000004    Lavadora 11 Kg  Eletrodomésticos       765.32       765.32   
4  SKU-0000005    Lavadora 11 Kg  Eletrodomésticos       459.89       459.89   

      nome_vendedor data_venda  
0      Ana Teixeira 2012-10-04  
1      Josias Silva 2012-01-01  
2      Josias Silva 2012-02-02  
3  Mateus Gonçalves 2012-03-03  
4     Artur Moreira 2012-04-04  


In [ ]:
# ## Etapa 3: Criação da Tabela no Banco de Dados

In [62]:
# %%
def criar_tabela_vendas(conn):
    """Cria a tabela vendas_final no banco de dados"""
    try:
        cursor = conn.cursor()
        
        # SQL para criar a tabela
        create_table_query = """
        DROP TABLE IF EXISTS vendas_final;
        CREATE TABLE vendas_final (
            id SERIAL PRIMARY KEY,
            cod_produto VARCHAR(20) NOT NULL,
            nome_produto VARCHAR(100) NOT NULL,
            categoria VARCHAR(50) NOT NULL,
            valor_venda NUMERIC(10, 2) NOT NULL,
            venda_final NUMERIC(10, 2) NOT NULL,
            nome_vendedor VARCHAR(100) NOT NULL,
            data_venda DATE NOT NULL
        );
        """
        
        cursor.execute(create_table_query)
        conn.commit()
        print("Tabela 'vendas_final' criada com sucesso!")
        
    except Exception as e:
        conn.rollback()
        print(f"Erro ao criar tabela: {e}")
    finally:
        cursor.close()

In [63]:
# Executando a criação da tabela
if conn is not None:
    criar_tabela_vendas(conn)

Tabela 'vendas_final' criada com sucesso!


In [ ]:
# ## Etapa 4: Inserção dos Dados no Banco

In [64]:
# %%
def inserir_dados(conn, df):
    """Insere os dados do DataFrame na tabela vendas_final"""
    try:
        cursor = conn.cursor()
        
        # Preparar a query de inserção
        insert_query = """
        INSERT INTO vendas_final 
        (cod_produto, nome_produto, categoria, valor_venda, venda_final, nome_vendedor, data_venda)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """
        
        # Converter o DataFrame para uma lista de tuplas
        data_tuples = [
            tuple(x) for x in df[[
                'cod_produto', 'nome_produto', 'categoria', 
                'valor_venda', 'venda_final', 'nome_vendedor', 'data_venda'
            ]].to_numpy()
        ]
        
        # Executar o batch insert
        execute_batch(cursor, insert_query, data_tuples)
        conn.commit()
        print(f"Dados inseridos com sucesso! Total de registros: {len(df)}")
        
    except Exception as e:
        conn.rollback()
        print(f"Erro ao inserir dados: {e}")
    finally:
        cursor.close()


In [65]:
 ##Executando a inserção dos dados
if conn is not None:
    inserir_dados(conn, df_final)


Dados inseridos com sucesso! Total de registros: 457


In [ ]:
# ## Etapa 5: Consultas Analíticas

In [67]:
def executar_consulta(conn, query, params=None):
    """Executa uma consulta SQL e retorna os resultados"""
    try:
        cursor = conn.cursor()
        
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
            
        # Obter os nomes das colunas
        colnames = [desc[0] for desc in cursor.description]
        
        # Obter os resultados
        resultados = cursor.fetchall()
        
        return colnames, resultados
        
    except Exception as e:
        print(f"Erro ao executar consulta: {e}")
        return None, None
    finally:
        cursor.close()

In [68]:
# %%
# Consulta 1: Total de vendas por categoria
query1 = """
SELECT 
    categoria,
    ROUND(SUM(venda_final), 2) as total_vendas
FROM 
    vendas_final
GROUP BY 
    categoria
ORDER BY 
    total_vendas DESC;
"""

In [69]:
 ##Executando a consulta
if conn is not None:
    colnames1, resultados1 = executar_consulta(conn, query1)
    
    if resultados1:
        print("\nTotal de vendas por categoria:")
        print("{:<20} {:<15}".format("Categoria", "Total Vendas"))
        for row in resultados1:
            print("{:<20} R$ {:<15.2f}".format(row[0], row[1]))



Total de vendas por categoria:
Categoria            Total Vendas   
Eletrodomésticos     R$ 194764.11      
Celulares            R$ 118718.40      
Eletrônicos          R$ 48329.55       
Eletroportáteis      R$ 19055.45       


In [70]:
# Consulta 2: Top 3 vendedores
query2 = """
SELECT 
    nome_vendedor,
    ROUND(SUM(venda_final), 2) as total_vendas,
    COUNT(*) as qtd_vendas
FROM 
    vendas_final
GROUP BY 
    nome_vendedor
ORDER BY 
    total_vendas DESC
LIMIT 3;
"""

In [71]:
# Executando a consulta
if conn is not None:
    colnames2, resultados2 = executar_consulta(conn, query2)
    
    if resultados2:
        print("\nTop 3 vendedores por valor total de vendas:")
        print("{:<20} {:<15} {:<10}".format("Vendedor", "Total Vendas", "Qtd Vendas"))
        for row in resultados2:
            print("{:<20} R$ {:<15.2f} {:<10}".format(row[0], row[1], row[2]))


Top 3 vendedores por valor total de vendas:
Vendedor             Total Vendas    Qtd Vendas
André Pereira        R$ 94423.89        84        
Maria Fernandes      R$ 59811.26        77        
Artur Moreira        R$ 54927.79        57        


In [72]:
# Consulta 3: Média de valor final de venda
query3 = """
SELECT 
    ROUND(AVG(venda_final), 2) as media_venda_final
FROM 
    vendas_final;
"""

In [73]:
# Executando a consulta
if conn is not None:
    colnames3, resultados3 = executar_consulta(conn, query3)
    
    if resultados3:
        print("\nMédia do valor final de venda:")
        print("R$ {:.2f}".format(resultados3[0][0]))



Média do valor final de venda:
R$ 833.41


In [ ]:
# ## Etapa 6: Fechamento da Conexão


In [74]:
# Fechando a conexão com o banco de dados
if conn is not None:
    conn.close()
    print("\nConexão com o banco de dados encerrada.")


Conexão com o banco de dados encerrada.
